# 计算摄影学中学的图像去噪
前面已经介绍过 均值滤波、高斯滤波、中值滤波、双边滤波、时域滤波、频域滤波、小波去噪等方法，这里介绍一些其他的去噪方法。
## 非局部均值去噪
这里指的是非局部均值去噪（Non-local Means Denoising）算法，它是一种基于图像自相似性的去噪方法。它的基本思想是：对于图像中的某个像素点，以它为中心的邻域内的像素值应该与图像中其他地方的像素值相似。因此，我们可以通过对图像中所有像素点进行求平均的方法来消除噪声。这种方法的一个缺点是计算量太大，因为对于图像中的每个像素点，都需要对整幅图像进行搜索。为了减少计算量，我们可以对图像进行分块，然后对每个分块内的像素点进行求平均。这样做的结果是，图像中的每个像素点都会有多个不同的求平均结果，这些结果会根据像素点的位置进行加权平均，从而得到最终的结果。这种方法的一个优点是，它可以保留图像的边缘信息。

## cv.fastNlMeansDenoising()-处理单个灰度图像

## **cv.fastNlMeansDenoisingColored()-处理彩色图像。**
    
```
cv.fastNlMeansDenoisingColored(src, dst, h, hForColorComponents, templateWindowSize, searchWindowSize)
参数解释:
- h: 决定过滤器强度。h值高可以很好的去除噪声，但也会把图像的细节抹去。h值低则会保留更多的细节，但是不能很好的去除噪声。默认值为10。
- hForColorComponents: 与h相同，但使用与彩色图像。默认值为10。
- templateWindowSize: 奇数。建议值为7。它是用于计算像素邻域的大小。这些像素用于计算具有相似权重的像素的均值。根据参数h的大小，通常应该在1到10之间选择该值。
- searchWindowSize: 奇数。建议值为21。它是用于在每个像素周围搜索邻域的大小以估计噪声像素的均值。与上面的templateWindowSize相同，但应小于它。通常，它的值在templateWindowSize的一半到templateWindowSize之间。

返回值:
- dst: 输出图像。
    
```

In [10]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import _tools

img = cv.imread('../data/opencv-logo.png') 
#添加高斯噪声
gauss_noise = _tools.AddGaussNoise(img,0.35)
#有效果不是很明显哈哈
dst = cv.fastNlMeansDenoisingColored(gauss_noise,None,10,10,7,21)
# 拼接图像
total = np.concatenate((img, gauss_noise, dst), axis=1)
cv.imshow('total', total)
cv.waitKey(0)
cv.destroyAllWindows()


## **cv.fastNlMeansDenoisingMulti()-处理在短时间内捕获的图像序列（灰度图像）**
        
```
cv.fastNlMeansDenoisingMulti(srcImgs, imgToDenoiseIndex, temporalWindowSize, dst, h, templateWindowSize, searchWindowSize)
参数解释:
- srcImgs: 输入图像序列。它应该是一个3D NumPy数组，其中的每一维都是一个图像。例如，如果您的视频序列有5个元素，则src应该是一个5xHxW的NumPy数组，其中H和W是图像的高度和宽度。
- imgToDenoiseIndex: 序列中要去噪的图像的索引。在上面的例子中，您可以选择0到4之间的任何值。
- temporalWindowSize: 奇数。建议值为5。它是用于计算像素邻域的大小。这些像素用于计算具有相似权重的像素的均值。根据参数h的大小，通常应该在1到10之间选择该值。
- h: 决定过滤器强度。h值高可以很好的去除噪声，但也会把图像的细节抹去。h值低则会保留更多的细节，但是不能很好的去除噪声。默认值为10。
- templateWindowSize: 奇数。建议值为7。它是用于计算像素邻域的大小。这些像素用于计算具有相似权重的像素的均值。根据参数h的大小，通常应该在1到10之间选择该值。
- searchWindowSize: 奇数。建议值为21。它是用于在每个像素周围搜索邻域的大小以估计噪声像素的均值。与上面的templateWindowSize相同，但应小于它。通常，它的值在templateWindowSize的一半到templateWindowSize之间。
- dst: 输出图像。


    
```
    
## cv.fastNlMeansDenoisingColoredMulti()-与上面相同，但用于彩色图像。

In [20]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
cap = cv.VideoCapture('../data/vtest.avi')
# 创建5个帧的列表
# cap.read()[1]是读取视频的每一帧
img = [cap.read()[1] for i in range(5)]
# 将所有转化为灰度
gray = [cv.cvtColor(i, cv.COLOR_BGR2GRAY) for i in img]
# 将所有转化为float64
gray = [np.float64(i) for i in gray]
# 创建方差为25的噪声     *gray[1].shape上的*是解包操作 *15是标准差
noise = np.random.randn(*gray[1].shape)*5

# 在图像上添加噪声  
noisy = [i+noise for i in gray]

# 转化为unit8    np.clip(i,0,255)是将i限制在0-255之间
noisy = [np.uint8(np.clip(i,0,255)) for i in noisy]

# 对第三帧进行降噪
dst = cv.fastNlMeansDenoisingMulti(noisy, 2, 5, None, 4, 7, 35)

# 拼接图像
# total = np.concatenate((gray[2], noisy[2], dst), axis=1)
# cv.imshow('total', total)
cv.imshow('gray[2]', gray[2])
cv.imshow('noisy[2]', noisy[2])
cv.imshow('dst', dst)
cv.waitKey(0)
cv.destroyAllWindows()
